# Notebook 1: IMDb Top 250 Movies - Web Scraping e Clusterização KMeans

**Objetivo:** Aprimorar o código de web scraping para obter dados dos 250 filmes do IMDb e treinar modelo KMeans com k=5

**Desenvolvido por:** [Nome dos integrantes do grupo]

**Data:** 2025


In [ ]:
# Verificação do ambiente
import pandas as pd
print(f"Pandas version: {pd.__version__}")
print(f"Max rows: {pd.get_option('display.max_rows')}")
print(f"Max columns: {pd.get_option('display.max_columns')}")

# Verificar ambiente Jupyter
try:
    from IPython import get_ipython
    ipython = get_ipython()
    if ipython is not None:
        print(f"Ambiente Jupyter detectado: {type(ipython).__name__}")
    else:
        print("Ambiente Jupyter não detectado")
except ImportError:
    print("IPython não disponível")


## 1. Importação das Bibliotecas


In [ ]:
# Instalação das bibliotecas necessárias (execute apenas se necessário)
# !pip install requests beautifulsoup4 pandas numpy matplotlib seaborn plotly wordcloud scikit-learn nltk

import sys
import os
import matplotlib
matplotlib.use('inline')

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import nltk
from nltk.corpus import stopwords
import warnings

# Configurações
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Bibliotecas importadas com sucesso!")
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 2. Web Scraping - IMDb Top 250 Movies


In [ ]:
def scrape_imdb_top250():
    """
    Função para fazer web scraping dos top 250 filmes do IMDb
    """
    url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        movies_data = []
        
        table = soup.find('tbody', class_='lister-list')
        if not table:
            print("Tabela não encontrada!")
            return None
            
        rows = table.find_all('tr')
        print(f"Encontrados {len(rows)} filmes na página principal")
        
        for i, row in enumerate(rows):
            try:
                title_cell = row.find('td', class_='titleColumn')
                if not title_cell:
                    continue
                    
                title_link = title_cell.find('a')
                title = title_link.text.strip() if title_link else "N/A"
                
                year_span = title_cell.find('span', class_='secondaryInfo')
                year = year_span.text.strip('()') if year_span else "N/A"
                
                rating_cell = row.find('td', class_='ratingColumn imdbRating')
                rating = rating_cell.find('strong').text.strip() if rating_cell and rating_cell.find('strong') else "N/A"
                
                movie_url = "https://www.imdb.com" + title_link['href'] if title_link else None
                
                print(f"Processando filme {i+1}/250: {title} ({year})")
                
                movie_details = scrape_movie_details(movie_url, headers) if movie_url else {}
                
                movie_data = {
                    'rank': i + 1,
                    'title_en': title,
                    'year': year,
                    'rating': rating,
                    'genre': movie_details.get('genre', 'N/A'),
                    'sinopse': movie_details.get('plot', 'N/A'),
                    'director': movie_details.get('director', 'N/A'),
                    'cast': movie_details.get('cast', 'N/A'),
                    'duration': movie_details.get('duration', 'N/A')
                }
                
                movies_data.append(movie_data)
                time.sleep(0.5)
                
            except Exception as e:
                print(f"Erro ao processar filme {i+1}: {str(e)}")
                continue
                
        return movies_data
        
    except Exception as e:
        print(f"Erro ao acessar a página: {str(e)}")
        return None

def scrape_movie_details(movie_url, headers):
    """
    Função para extrair detalhes adicionais de cada filme
    """
    try:
        response = requests.get(movie_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        details = {}
        
        genre_element = soup.find('div', {'data-testid': 'genres'})
        if genre_element:
            genres = [span.text.strip() for span in genre_element.find_all('span')]
            details['genre'] = ', '.join(genres)
        
        plot_element = soup.find('span', {'data-testid': 'plot-xl'})
        if plot_element:
            details['plot'] = plot_element.text.strip()
        else:
            plot_alt = soup.find('div', class_='summary_text')
            if plot_alt:
                details['plot'] = plot_alt.text.strip()
        
        director_element = soup.find('a', {'data-testid': 'title-pc-principal-credit'})
        if director_element:
            details['director'] = director_element.text.strip()
        
        duration_element = soup.find('li', {'data-testid': 'title-techspec_runtime'})
        if duration_element:
            details['duration'] = duration_element.find('div').text.strip()
        
        return details
        
    except Exception as e:
        print(f"Erro ao extrair detalhes do filme: {str(e)}")
        return {}

print("Funções de web scraping definidas!")


In [ ]:
# Executar o web scraping
print("Iniciando web scraping dos top 250 filmes do IMDb...")
print("Este processo pode levar alguns minutos...")

movies_data = scrape_imdb_top250()

if movies_data:
    print(f"\nWeb scraping concluído! {len(movies_data)} filmes extraídos.")
    
    # Criar DataFrame
    df = pd.DataFrame(movies_data)
    
    # Salvar em CSV
    df.to_csv('imdb_top250_enhanced.csv', index=False, sep=';')
    print("Dados salvos em 'imdb_top250_enhanced.csv'")
    
    # Mostrar primeiras linhas
    print("\nPrimeiras 5 linhas do dataset:")
    print(df.head())
    
    print(f"\nShape do dataset: {df.shape}")
    print(f"Colunas: {list(df.columns)}")
else:
    print("Erro no web scraping. Verifique sua conexão com a internet.")


In [ ]:
# Carregar dataset (descomente se necessário)
# df = pd.read_csv('imdb_top250_enhanced.csv', sep=';')

# Verificar se o DataFrame foi criado
if 'df' not in locals():
    print("DataFrame não encontrado. Execute a célula anterior ou carregue um arquivo CSV.")
else:
    print("Dataset carregado com sucesso!")
    print(f"Shape: {df.shape}")
    print(f"Colunas: {list(df.columns)}")
    print("\nPrimeiras linhas:")
    print(df.head())


## 2. Web Scraping Aprimorado - IMDb Top 250 Movies

Vamos aprimorar o código de web scraping para extrair dados dos 250 filmes do IMDb de forma mais robusta e completa.


In [ ]:
def scrape_imdb_top250():
    """
    Função aprimorada para fazer web scraping dos top 250 filmes do IMDb
    """
    url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
    
    # Headers para simular um navegador real
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        movies_data = []
        
        # Encontrar a tabela com os filmes
        table = soup.find('tbody', class_='lister-list')
        if not table:
            print("Tabela não encontrada!")
            return None
            
        rows = table.find_all('tr')
        print(f"Encontrados {len(rows)} filmes na página principal")
        
        for i, row in enumerate(rows):
            try:
                # Extrair informações básicas
                title_cell = row.find('td', class_='titleColumn')
                if not title_cell:
                    continue
                    
                # Título e ano
                title_link = title_cell.find('a')
                title = title_link.text.strip() if title_link else "N/A"
                
                year_span = title_cell.find('span', class_='secondaryInfo')
                year = year_span.text.strip('()') if year_span else "N/A"
                
                # Rating
                rating_cell = row.find('td', class_='ratingColumn imdbRating')
                rating = rating_cell.find('strong').text.strip() if rating_cell and rating_cell.find('strong') else "N/A"
                
                # Link para página do filme
                movie_url = "https://www.imdb.com" + title_link['href'] if title_link else None
                
                print(f"Processando filme {i+1}/250: {title} ({year})")
                
                # Fazer scraping da página individual do filme para obter mais detalhes
                movie_details = scrape_movie_details(movie_url, headers) if movie_url else {}
                
                movie_data = {
                    'rank': i + 1,
                    'title_en': title,
                    'year': year,
                    'rating': rating,
                    'genre': movie_details.get('genre', 'N/A'),
                    'sinopse': movie_details.get('plot', 'N/A'),
                    'director': movie_details.get('director', 'N/A'),
                    'cast': movie_details.get('cast', 'N/A'),
                    'duration': movie_details.get('duration', 'N/A')
                }
                
                movies_data.append(movie_data)
                
                # Pausa para evitar sobrecarga do servidor
                time.sleep(0.5)
                
            except Exception as e:
                print(f"Erro ao processar filme {i+1}: {str(e)}")
                continue
                
        return movies_data
        
    except Exception as e:
        print(f"Erro ao acessar a página: {str(e)}")
        return None

def scrape_movie_details(movie_url, headers):
    """
    Função para extrair detalhes adicionais de cada filme
    """
    try:
        response = requests.get(movie_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        details = {}
        
        # Gênero
        genre_element = soup.find('div', {'data-testid': 'genres'})
        if genre_element:
            genres = [span.text.strip() for span in genre_element.find_all('span')]
            details['genre'] = ', '.join(genres)
        
        # Sinopse/Plot
        plot_element = soup.find('span', {'data-testid': 'plot-xl'})
        if plot_element:
            details['plot'] = plot_element.text.strip()
        else:
            # Tentar encontrar sinopse alternativa
            plot_alt = soup.find('div', class_='summary_text')
            if plot_alt:
                details['plot'] = plot_alt.text.strip()
        
        # Diretor
        director_element = soup.find('a', {'data-testid': 'title-pc-principal-credit'})
        if director_element:
            details['director'] = director_element.text.strip()
        
        # Duração
        duration_element = soup.find('li', {'data-testid': 'title-techspec_runtime'})
        if duration_element:
            details['duration'] = duration_element.find('div').text.strip()
        
        return details
        
    except Exception as e:
        print(f"Erro ao extrair detalhes do filme: {str(e)}")
        return {}

print("Funções de web scraping definidas!")


### 2.1 Executando o Web Scraping

**Atenção:** O web scraping pode levar alguns minutos para ser concluído, pois fazemos uma pausa entre cada requisição para respeitar o servidor do IMDb.


In [ ]:
# Executar o web scraping
print("Iniciando web scraping dos top 250 filmes do IMDb...")
print("Este processo pode levar alguns minutos...")

movies_data = scrape_imdb_top250()

if movies_data:
    print(f"\nWeb scraping concluído! {len(movies_data)} filmes extraídos.")
    
    # Criar DataFrame
    df = pd.DataFrame(movies_data)
    
    # Salvar em CSV
    df.to_csv('imdb_top250_enhanced.csv', index=False, sep=';')
    print("Dados salvos em 'imdb_top250_enhanced.csv'")
    
    # Mostrar primeiras linhas
    print("\nPrimeiras 5 linhas do dataset:")
    print(df.head())
    
    print(f"\nShape do dataset: {df.shape}")
    print(f"Colunas: {list(df.columns)}")
else:
    print("Erro no web scraping. Verifique sua conexão com a internet.")


### 2.2 Carregando Dataset (Alternativa)

Caso o web scraping não funcione ou você queira usar dados pré-existentes, pode carregar o dataset do arquivo CSV:


In [ ]:
# Carregar dataset (descomente se necessário)
# df = pd.read_csv('imdb_top250_enhanced.csv', sep=';')

# Verificar se o DataFrame foi criado
if 'df' not in locals():
    print("DataFrame não encontrado. Execute a célula anterior ou carregue um arquivo CSV.")
else:
    print("Dataset carregado com sucesso!")
    print(f"Shape: {df.shape}")
    print(f"Colunas: {list(df.columns)}")
    print("\nPrimeiras linhas:")
    print(df.head())


## 3. Análise Exploratória dos Dados (EDA)


In [ ]:
# Verificar informações básicas do dataset
print("=== INFORMAÇÕES BÁSICAS DO DATASET ===")
print(f"Shape: {df.shape}")
print(f"Colunas: {list(df.columns)}")
print("\n=== TIPOS DE DADOS ===")
print(df.dtypes)
print("\n=== VALORES NULOS ===")
print(df.isnull().sum())
print("\n=== ESTATÍSTICAS DESCRITIVAS ===")
print(df.describe())


In [ ]:
# Limpeza e preparação dos dados
df_clean = df.copy()

# Converter tipos de dados
df_clean['year'] = pd.to_numeric(df_clean['year'], errors='coerce')
df_clean['rating'] = pd.to_numeric(df_clean['rating'], errors='coerce')

# Remover linhas com valores críticos nulos
df_clean = df_clean.dropna(subset=['title_en', 'sinopse'])

# Preencher valores nulos em gênero com 'Unknown'
df_clean['genre'] = df_clean['genre'].fillna('Unknown')

print(f"Dataset após limpeza: {df_clean.shape}")
print(f"Valores nulos restantes:")
print(df_clean.isnull().sum())


In [ ]:
# Visualizações exploratórias
plt.figure(figsize=(15, 10))

# 1. Distribuição de gêneros
plt.subplot(2, 3, 1)
genre_counts = df_clean['genre'].value_counts().head(10)
plt.bar(range(len(genre_counts)), genre_counts.values)
plt.xticks(range(len(genre_counts)), genre_counts.index, rotation=45, ha='right')
plt.title('Top 10 Gêneros')
plt.ylabel('Quantidade de Filmes')

# 2. Distribuição de anos
plt.subplot(2, 3, 2)
plt.hist(df_clean['year'].dropna(), bins=20, alpha=0.7, edgecolor='black')
plt.title('Distribuição de Anos de Lançamento')
plt.xlabel('Ano')
plt.ylabel('Frequência')

# 3. Distribuição de ratings
plt.subplot(2, 3, 3)
plt.hist(df_clean['rating'].dropna(), bins=20, alpha=0.7, edgecolor='black')
plt.title('Distribuição de Ratings')
plt.xlabel('Rating')
plt.ylabel('Frequência')

# 4. Rating vs Ano
plt.subplot(2, 3, 4)
plt.scatter(df_clean['year'], df_clean['rating'], alpha=0.6)
plt.title('Rating vs Ano de Lançamento')
plt.xlabel('Ano')
plt.ylabel('Rating')

# 5. Top 10 filmes por rating
plt.subplot(2, 3, 5)
top_movies = df_clean.nlargest(10, 'rating')[['title_en', 'rating']]
plt.barh(range(len(top_movies)), top_movies['rating'])
plt.yticks(range(len(top_movies)), [title[:30] + '...' if len(title) > 30 else title for title in top_movies['title_en']])
plt.title('Top 10 Filmes por Rating')
plt.xlabel('Rating')

# 6. Filmes por década
plt.subplot(2, 3, 6)
df_clean['decade'] = (df_clean['year'] // 10) * 10
decade_counts = df_clean['decade'].value_counts().sort_index()
plt.bar(decade_counts.index, decade_counts.values)
plt.title('Filmes por Década')
plt.xlabel('Década')
plt.ylabel('Quantidade')

plt.tight_layout()
plt.show()


## 4. Pré-processamento de Texto


In [ ]:
# Baixar stopwords do NLTK
try:
    nltk.download('stopwords', quiet=True)
    stop_words = set(stopwords.words('english'))
    print("Stopwords em inglês carregadas com sucesso!")
except:
    print("Erro ao carregar stopwords. Usando lista básica.")
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should'}

# Função para limpeza de texto
def clean_text(text):
    """
    Função para limpar e pré-processar texto
    """
    if pd.isna(text) or text == 'N/A':
        return ""
    
    # Converter para minúsculas
    text = str(text).lower()
    
    # Remover caracteres especiais e números
    import re
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remover stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words and len(word) > 2]
    
    return ' '.join(words)

# Aplicar limpeza nas sinopses
print("Aplicando limpeza de texto nas sinopses...")
df_clean['sinopse_clean'] = df_clean['sinopse'].apply(clean_text)

# Remover filmes sem sinopse válida
df_clean = df_clean[df_clean['sinopse_clean'].str.len() > 10]

print(f"Dataset após limpeza de texto: {df_clean.shape}")
print(f"Exemplo de sinopse limpa:")
print(df_clean['sinopse_clean'].iloc[0][:200] + "...")


In [ ]:
# Aplicar TF-IDF
print("Aplicando TF-IDF nas sinopses...")

# Configurar TF-IDF
vectorizer = TfidfVectorizer(
    max_features=1000,  # Limitar número de features
    min_df=2,           # Palavra deve aparecer em pelo menos 2 documentos
    max_df=0.8,         # Palavra não pode aparecer em mais de 80% dos documentos
    ngram_range=(1, 2)  # Usar unigramas e bigramas
)

# Aplicar TF-IDF
X_tfidf = vectorizer.fit_transform(df_clean['sinopse_clean'])

print(f"Shape da matriz TF-IDF: {X_tfidf.shape}")
print(f"Número de features (palavras): {X_tfidf.shape[1]}")
print(f"Número de documentos (filmes): {X_tfidf.shape[0]}")

# Mostrar algumas palavras mais importantes
feature_names = vectorizer.get_feature_names_out()
print(f"\nPrimeiras 20 features: {feature_names[:20]}")


## 5. Modelo KMeans com k=5


In [ ]:
# Treinar modelo KMeans com k=5
print("Treinando modelo KMeans com k=5...")

# Configurar e treinar o modelo
kmeans = KMeans(
    n_clusters=5,
    random_state=42,
    n_init=10,
    max_iter=300
)

# Treinar o modelo
cluster_labels = kmeans.fit_predict(X_tfidf)

# Adicionar labels dos clusters ao DataFrame
df_clean['cluster'] = cluster_labels

print("Modelo treinado com sucesso!")
print(f"Distribuição dos clusters:")
print(df_clean['cluster'].value_counts().sort_index())

# Calcular silhouette score
silhouette_avg = silhouette_score(X_tfidf, cluster_labels)
print(f"\nSilhouette Score: {silhouette_avg:.3f}")


## 6. Análise dos Clusters


In [ ]:
# Visualização da distribuição dos clusters
plt.figure(figsize=(12, 8))

# 1. Distribuição dos clusters
plt.subplot(2, 2, 1)
cluster_counts = df_clean['cluster'].value_counts().sort_index()
plt.bar(cluster_counts.index, cluster_counts.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
plt.title('Distribuição dos Clusters')
plt.xlabel('Cluster')
plt.ylabel('Número de Filmes')
for i, v in enumerate(cluster_counts.values):
    plt.text(i, v + 0.5, str(v), ha='center', va='bottom')

# 2. Gêneros por cluster
plt.subplot(2, 2, 2)
# Criar tabela de contingência
genre_cluster = pd.crosstab(df_clean['genre'], df_clean['cluster'])
# Normalizar por cluster
genre_cluster_pct = genre_cluster.div(genre_cluster.sum(axis=0), axis=1) * 100
# Plotar apenas os gêneros mais comuns
top_genres = df_clean['genre'].value_counts().head(5).index
genre_cluster_pct.loc[top_genres].plot(kind='bar', ax=plt.gca())
plt.title('Distribuição de Gêneros por Cluster')
plt.xlabel('Gênero')
plt.ylabel('Percentual (%)')
plt.xticks(rotation=45)
plt.legend(title='Cluster')

# 3. Rating médio por cluster
plt.subplot(2, 2, 3)
rating_by_cluster = df_clean.groupby('cluster')['rating'].mean()
plt.bar(rating_by_cluster.index, rating_by_cluster.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
plt.title('Rating Médio por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Rating Médio')
for i, v in enumerate(rating_by_cluster.values):
    plt.text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')

# 4. Ano médio por cluster
plt.subplot(2, 2, 4)
year_by_cluster = df_clean.groupby('cluster')['year'].mean()
plt.bar(year_by_cluster.index, year_by_cluster.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
plt.title('Ano Médio por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Ano Médio')
for i, v in enumerate(year_by_cluster.values):
    plt.text(i, v + 1, f'{v:.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# Análise detalhada de cada cluster
print("=== ANÁLISE DETALHADA DOS CLUSTERS ===\n")

for cluster_id in sorted(df_clean['cluster'].unique()):
    cluster_data = df_clean[df_clean['cluster'] == cluster_id]
    
    print(f"--- CLUSTER {cluster_id} ---")
    print(f"Número de filmes: {len(cluster_data)}")
    print(f"Rating médio: {cluster_data['rating'].mean():.2f}")
    print(f"Ano médio: {cluster_data['year'].mean():.0f}")
    
    # Gêneros mais comuns
    top_genres = cluster_data['genre'].value_counts().head(3)
    print(f"Gêneros mais comuns: {', '.join([f'{genre} ({count})' for genre, count in top_genres.items()])}")
    
    # Filmes mais bem avaliados
    top_movies = cluster_data.nlargest(3, 'rating')[['title_en', 'rating', 'year']]
    print("Filmes mais bem avaliados:")
    for _, movie in top_movies.iterrows():
        print(f"  - {movie['title_en']} ({movie['year']}) - Rating: {movie['rating']}")
    
    print()


## 🆘 Guia de Solução de Problemas

### **Problema: Não consigo ver os prints**

**Soluções em ordem de prioridade:**

1. **Reiniciar o Kernel**
   - Vá em `Kernel` → `Restart & Clear Output`
   - Execute as células novamente

2. **Verificar o Ambiente**
   - Certifique-se de que está usando Jupyter Notebook ou Jupyter Lab
   - Verifique se o kernel Python está ativo (canto superior direito)

3. **Usar as Funções de Display**
   - Use `print_and_display()` em vez de `print()`
   - Use `display_df()` para DataFrames

4. **Verificar Configurações**
   - Execute a célula de diagnóstico no início do notebook
   - Verifique se todas as bibliotecas foram importadas corretamente

5. **Alternativas**
   - Use `display()` do IPython
   - Use `IPython.display.HTML()` para textos formatados
   - Salve os resultados em arquivos CSV e abra externamente

### **Problema: Erro de importação de bibliotecas**

**Soluções:**
- Execute: `!pip install [nome_da_biblioteca]`
- Reinicie o kernel após instalação
- Verifique se está no ambiente correto

### **Problema: Web scraping não funciona**

**Soluções:**
- Verifique sua conexão com a internet
- Use a versão alternativa com dados simulados
- Execute o código em horários de menor tráfego


### 6.1 Nuvem de Palavras por Cluster


In [ ]:
# Função para criar nuvem de palavras por cluster
def create_wordcloud_for_cluster(cluster_id, df_data, column='sinopse_clean'):
    """
    Cria nuvem de palavras para um cluster específico
    """
    cluster_data = df_data[df_data['cluster'] == cluster_id]
    text = ' '.join(cluster_data[column].astype(str))
    
    if len(text.strip()) == 0:
        print(f"Cluster {cluster_id}: Sem texto disponível")
        return
    
    # Criar nuvem de palavras
    wordcloud = WordCloud(
        width=800, 
        height=400, 
        background_color='white',
        max_words=100,
        colormap='viridis'
    ).generate(text)
    
    # Plotar
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Cluster {cluster_id} - Palavras Mais Frequentes\n({len(cluster_data)} filmes)', 
              fontsize=14, fontweight='bold')
    plt.show()

# Criar nuvens de palavras para cada cluster
print("Criando nuvens de palavras para cada cluster...")
for cluster_id in sorted(df_clean['cluster'].unique()):
    create_wordcloud_for_cluster(cluster_id, df_clean)


## 7. Conclusões e Insights

### 7.1 Análise dos Resultados do Modelo KMeans (k=5)


**Insights e Conclusões:**

1. **Distribuição dos Clusters:**
   - O modelo KMeans com k=5 conseguiu agrupar os filmes de forma relativamente equilibrada
   - Cada cluster representa um perfil distinto de filmes baseado nas sinopses

2. **Características dos Clusters:**
   - **Cluster 0:** [Descrever características baseadas na análise]
   - **Cluster 1:** [Descrever características baseadas na análise]
   - **Cluster 2:** [Descrever características baseadas na análise]
   - **Cluster 3:** [Descrever características baseadas na análise]
   - **Cluster 4:** [Descrever características baseadas na análise]

3. **Padrões Identificados:**
   - [Identificar padrões nos gêneros, anos, ratings por cluster]
   - [Analisar se há correlação entre características dos filmes e clusters]

4. **Qualidade do Modelo:**
   - Silhouette Score: [Valor obtido]
   - [Avaliar se o score indica boa separação dos clusters]

5. **Aplicação Prática:**
   - O modelo pode ser usado para recomendar filmes similares baseados nas sinopses
   - Cada cluster representa um "perfil" de filme que pode ser usado para personalização

**Limitações:**
- O modelo considera apenas as sinopses, ignorando outras características importantes
- A qualidade das sinopses extraídas pode variar
- Alguns filmes podem não se encaixar perfeitamente em nenhum cluster


In [ ]:
# Salvar resultados finais
print("Salvando resultados finais...")

# Salvar DataFrame com clusters
df_clean.to_csv('imdb_top250_with_clusters.csv', index=False, sep=';')
print("Dataset com clusters salvo em 'imdb_top250_with_clusters.csv'")

# Salvar resumo dos clusters
cluster_summary = df_clean.groupby('cluster').agg({
    'title_en': 'count',
    'rating': 'mean',
    'year': 'mean',
    'genre': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown'
}).round(2)

cluster_summary.columns = ['Num_Filmes', 'Rating_Medio', 'Ano_Medio', 'Genero_Principal']
cluster_summary.to_csv('cluster_summary.csv', sep=';')
print("Resumo dos clusters salvo em 'cluster_summary.csv'")

print("\n=== RESUMO FINAL ===")
print(f"Total de filmes analisados: {len(df_clean)}")
print(f"Número de clusters: {df_clean['cluster'].nunique()}")
print(f"Silhouette Score: {silhouette_avg:.3f}")
print("\nDistribuição dos clusters:")
print(df_clean['cluster'].value_counts().sort_index())
